# Tutorial. Numerical solution of a partial differential equation using a collocation spectral method

In this notebook, a collocation spectral methods is used to solve an initial-value problem for a linear partial differential equation. Two algorithmic approaches are considered: one using the fast Fourier transform, the other involving multiplication by a differentiation matrix.

The following libraries will be needed, notably `fft` from <tt>scipy</tt>, notably for a computationally efficient implementation of the Chebyshev transform.

In [3]:
import numpy as np
import scipy as scp
from scipy import fft

# To draw matplotlib plots within this notebook.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

## Exercise 1. Differentiation in collocation spectral methods: fast Fourier transforms versus matrix multiplication.

When implementing a collocation spectral method for the solution of a partial differential equation problem, Fourier or chebyshev interpolation differentiation at the collocation points can be performed using matrix multiplication, rather than resorting to fast Fourier transform.

Fourier interpolation differentiation can indeed be represented by a matrix, called the *Fourier interpolation derivative matrix* $D_N$, such that the approximate value of the first derivative of a function $u$ at the collocation point $x_j=\frac{2\pi j}{N}$, $j=0,\dots,N-1$, with $N$ an *even* non-zero natural integer, is given by
$$
\sum_{l=0}^{N-1}\left(D_N^{(1)}\right)_{jl}u(x_l),
$$
where
$$
\left(D_N^{(1)}\right)_{jl}=\frac{1}{N}\sum_{k=-N/2+1}^{N/2}ike^{2ik\frac{(j-l)\pi}{N}}.
$$
Note here that the indices of the elements of the matrix start at value zero, like in the indexing of arrays in Python.

If $u$ is a real function, the last sum can be evaluated in closed form and, in practice, one has
$$
\left(D_N^{(1)}\right)_{jl}=\begin{cases}\frac{1}{2}(-1)^{j+l}\cot\left(\frac{(j-l)\pi}{N}\right)&\text{if }j\neq l,\\0&\text{if }j=l.\end{cases}
$$
One may notice that this matrix is a [Toeplitz matrix](https://en.wikipedia.org/wiki/Toeplitz_matrix). 

**Question.** Write a program for the differentiation of a discrete Fourier series approximation of the function $u(x)=\exp(\sin(x))$ using matrix multiplication (note the `linalg` library from <tt>scipy</tt> possesses a function for the construction of a Toeplitz matrix, see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.toeplitz.html)). Compare its accuracy and execution time to that of the code for differentiation based on the Fast Fourier transform, written in a previous tutorial, for several values of $N$ and using the `time` function in the `time` library (see the [documentation](https://docs.python.org/3/library/time.html#time.time)).

In [ ]:
import time

def build_Fourier_differentiation_matrix(N):
    row,col=np.zeros(N),np.zeros(N)
    col[1:]=0.5*(-1.)**np.arange(1,N)/np.tan(np.arange(1,N)*np.pi/N)
    row[1:]=col[N-1:0:-1]
    return scp.linalg.toeplitz(col,row)


N_values=2**np.arange(3,16)
error_matrix_multiply=np.zeros(N_values.size)
error_fft=np.zeros(N_values.size)
computational_time_matrix_multiply=np.zeros(N_values.size)
computational_time_fft=np.zeros(N_values.size)

for i,N in enumerate(N_values):
    x=np.linspace(0,2*np.pi,N,endpoint=False)
    u=np.exp(np.sin(x))
    uprime=np.cos(x)*u

    start_time=time.time()
    D=build_Fourier_differentiation_matrix(N)
    Du=D@u
    computational_time_matrix_multiply[i]=time.time()-start_time
    error_matrix_multiply[i]=np.linalg.norm(uprime-Du,np.inf)

    start_time=time.time()
    u_tilde=scp.fft.fft(u)
    weights=1j*np.zeros(N)
    weights[0:N//2]=1j*np.arange(0,N//2)
    weights[N//2+1:] = 1j*np.arange(-N//2+1,0,1)
    Du=scp.fft.ifft(weights * u_tilde).real
    computational_time_fft[i]=time.time()-start_time
    error_fft[i]=np.linalg.norm(uprime-Du,np.inf)

fig,axs=plt.subplots(1,2,figsize=(12,5))
axs[0].loglog(N_values,error_matrix_multiply,label='matrix multiply')
axs[0].loglog(N_values,error_fft,label='fast Fourier transform')
axs[0].set_xlabel(r'$N$')
axs[0].set_ylabel('supremum norm of the error')
axs[0].legend()

axs[1].loglog(N_values,computational_time_matrix_multiply,label='matrix multiply')
axs[1].loglog(N_values,computational_time_fft,label='fast Fourier transform')
axs[1].set_xlabel(r'$N$')
axs[1].set_ylabel('execution time')
axs[1].legend()

fig.tight_layout()

**Answer.** A rapid convergence is observed, typical of the spectral accuracy of the method for smooth functions. The fact that error grows for larger values of $N$ is the result of the bad conditioning of the derivative operator in the presence of roundoff errors. The approach using the fast Fourier transform appears to be the most accurate of the two and it is the fastest for larger values of $N$. Indeed, it scales asymptotically as $O(N\log_2(N))$, instead of $O(N^2)$ for the alternative approach, as can be seen by plotting lines with these reference slopes.

In [ ]:
plt.loglog(N_values,computational_time_matrix_multiply,label='matrix multiply')
plt.loglog(N_values,computational_time_fft,label='fast Fourier transform')
plt.loglog(N_values,1e-7*N_values**2,linestyle='dashed',label=r'$N^2$')
plt.loglog(N_values,1e-9*N_values*np.log(N_values),linestyle='dashdot',label=r'$N\log(N)$')
plt.xlabel(r'$N$')
plt.ylabel('execution time')
plt.legend()

For non periodic functions, the Chebyshev interpolation differentiation can be also achieved using a matrix. In this case, the first-order differentiation matrix can be written in closed-form, and one has
$$
\left(D_N^{(1)}\right)_{jl}=\begin{cases}\dfrac{2N^2+1}{6}&j=l=0,\\-\dfrac{x_j}{2(1-{x_j}^2)}&1\leq j,l\leq N-1,\ j=l\\\dfrac{c_i}{c_j}\dfrac{(-1)^{j+l}}{x_j-x_l}&1\leq j,l\leq N-1,\ j\neq l,\\-\dfrac{2N^2+1}{6}&j=l=N,\end{cases}
$$
where
$$
c_j=\begin{cases}2&\text{if }j=0\text{ or }N,\\1,&\text{if }j=1,\dots,N-1,\end{cases}\text{ and }x_j=\cos\left(\frac{\pi j}{N}\right),\ j=0,\dots,N.
$$

**Question.** Write a program for the differentiation of a discrete Chebyshev series approximation of the function $u(x)=\exp(-x^2)$ using matrix multiplication. Compare its accuracy and execution time to that of the code for differentiation based on the Fast Fourier transform, written in a previous tutorial, for several values of $N$.

In [ ]:
def build_Chebyshev_differentiation_matrix(N):
    c=np.ones(N+1)
    c[0],c[N]=2.,2.
    c=c*(-1.)**np.arange(0,N+1)
    c=c.reshape(N+1,1)
    X=np.tile(x.reshape(N+1,1),(1,N+1))
    D=np.dot(c,1./c.T)/(X-X.T+np.eye(N+1)) # off-diagonal entries
    D=D-np.diag(D.sum(axis=1)) # diagonal entries
    return D

N_values=2**np.arange(3,15)
error_matrix_multiply=np.zeros(N_values.size)
error_fft=np.zeros(N_values.size)
computational_time_matrix_multiply=np.zeros(N_values.size)
computational_time_fft=np.zeros(N_values.size)

for i,N in enumerate(N_values):
    x=np.cos(np.pi*np.arange(0,N+1)/N)

    u=np.exp(-x**2)
    uprime=-2.*x*u
    Du=np.zeros(N+1)

    start_time=time.time()
    D=build_Chebyshev_differentiation_matrix(N)
    Du=D@u
    computational_time_matrix_multiply[i]=time.time()-start_time
    error_matrix_multiply[i]=np.linalg.norm(uprime-Du,np.inf)

    start_time=time.time()
    u_tilde=scp.fft.dct(u,type=1)/N
    u_tilde[0]=u_tilde[0]/2.
    u_tilde[-1]=u_tilde[-1]/2.
    k_values=np.arange(N+1)
    Du[0]=np.dot(k_values**2,u_tilde)
    Du[1:-1]=N*scp.fft.idst(k_values[1:-1]*u_tilde[1:-1],type=1)/np.sqrt(1.-(np.cos(np.pi*k_values/N)[1:-1])**2)
    Du[-1]=np.dot(k_values**2*np.power(-np.ones(N+1),k_values+1),u_tilde)
    computational_time_fft[i]=time.time()-start_time
    error_fft[i]=np.linalg.norm(uprime-Du,np.inf)

fig,axs=plt.subplots(1,2,figsize=(12,5))
axs[0].loglog(N_values,error_matrix_multiply,label='matrix multiply')
axs[0].loglog(N_values,error_fft,label='fast Fourier transform')
axs[0].set_xlabel(r'$N$')
axs[0].set_ylabel('supremum norm of the error')
axs[0].legend()

axs[1].loglog(N_values,computational_time_matrix_multiply,label='matrix multiply')
axs[1].loglog(N_values,computational_time_fft,label='fast Fourier transform')
axs[1].set_xlabel(r'$N$')
axs[1].set_ylabel('execution time')
axs[1].legend()

fig.tight_layout()

**Answer.** While computations take more time, the observed behaviours of the two approaches for the Chebyshev interpolation differentiation is close to what it is for the Fourier interpolation, due to similar conditioning and algorithmic complexities ($O(N^2)$ and $O(N\log_2(N))$, respectively).

In [ ]:
plt.loglog(N_values,computational_time_matrix_multiply,label='matrix multiply')
plt.loglog(N_values,computational_time_fft,label='fast Fourier transform')
plt.loglog(N_values,1e-6*N_values**2,linestyle='dashed',label=r'$N^2$')
plt.loglog(N_values,1e-8*N_values*np.log(N_values),linestyle='dashdot',label=r'$N\log(N)$')
plt.xlabel(r'$N$')
plt.ylabel('execution time')
plt.legend()

Note that, using trigonometric identities, alternative expressions reducing the round-off error which results from the subtraction of nearly equal numbers can be found for the Chebyshev derivative matrices. For the first-order differentiation matrix, one has for instance
$$
\left(D_N^{(1)}\right)_{jl}=\begin{cases}\dfrac{2N^2+1}{6}&j=l=0,\\-\dfrac{x_j}{2(\sin(j\pi/N))^2}&1\leq j,l\leq N-1,\ j=l\\-\dfrac{c_i}{2c_j}\dfrac{(-1)^{j+l}}{\sin((j+l)\pi/2N)\sin((j-l)\pi/2N)}&1\leq j,l\leq N-1,\ j\neq l,\\-\dfrac{2N^2+1}{6}&j=l=N.\end{cases}
$$

In [ ]:
def build_Chebyshev_differentiation_matrix_alternate(N):
    c=np.ones(N+1)
    c[0],c[N]=2.,2.
    c=c*(-1.)**np.arange(0,N+1)
    c=c.reshape(N+1,1)
    X=np.tile(x.reshape(N+1,1),(1,N+1))
    J=np.tile(np.arange(0,N+1).reshape(N+1,1),(1,N+1))
    D=np.dot(c,1./c.T)/(-2.*np.sin(0.5*(J+J.T)*np.pi/N)*np.sin(0.5*(J-J.T)*np.pi/N)+np.eye(N+1)) # off-diagonal entries
    D=D-np.diag(D.sum(axis=1)) # diagonal entries
    return D

N_values=2**np.arange(3,15)
error_matrix_multiply=np.zeros(N_values.size)
error_matrix_multiply_alternate=np.zeros(N_values.size)
error_fft=np.zeros(N_values.size)
computational_time_matrix_multiply=np.zeros(N_values.size)
computational_time_matrix_multiply_alternate=np.zeros(N_values.size)
computational_time_fft=np.zeros(N_values.size)

for i,N in enumerate(N_values):
    x=np.cos(np.pi*np.arange(0,N+1)/N)

    u=np.exp(-x**2)
    uprime=-2.*x*u
    Du=np.zeros(N+1)

    start_time=time.time()
    D=build_Chebyshev_differentiation_matrix(N)
    Du=D@u
    computational_time_matrix_multiply[i]=time.time()-start_time
    error_matrix_multiply[i]=np.linalg.norm(uprime-Du,np.inf)

    start_time=time.time()
    D=build_Chebyshev_differentiation_matrix_alternate(N)
    Du=D@u
    computational_time_matrix_multiply_alternate[i]=time.time()-start_time
    error_matrix_multiply_alternate[i]=np.linalg.norm(uprime-Du,np.inf)

    start_time=time.time()
    u_tilde=scp.fft.dct(u,type=1)/N
    u_tilde[0]=u_tilde[0]/2.
    u_tilde[-1]=u_tilde[-1]/2.
    k_values=np.arange(N+1)
    Du[0]=np.dot(k_values**2,u_tilde)
    Du[1:-1]=N*scp.fft.idst(k_values[1:-1]*u_tilde[1:-1],type=1)/np.sqrt(1.-(np.cos(np.pi*k_values/N)[1:-1])**2)
    Du[-1]=np.dot(k_values**2*np.power(-np.ones(N+1),k_values+1),u_tilde)
    computational_time_fft[i]=time.time()-start_time
    error_fft[i]=np.linalg.norm(uprime-Du,np.inf)

fig,axs=plt.subplots(1,2,figsize=(12,5))
axs[0].loglog(N_values,error_matrix_multiply,label='matrix multiply')
axs[0].loglog(N_values,error_matrix_multiply_alternate,label='matrix multiply alt.')
axs[0].loglog(N_values,error_fft,label='fast Fourier transform')
axs[0].set_xlabel(r'$N$')
axs[0].set_ylabel('supremum norm of the error')
axs[0].legend()

axs[1].loglog(N_values,computational_time_matrix_multiply,label='matrix multiply')
axs[1].loglog(N_values,computational_time_matrix_multiply_alternate,label='matrix multiply alt.')
axs[1].loglog(N_values,computational_time_fft,label='fast Fourier transform')
axs[1].set_xlabel(r'$N$')
axs[1].set_ylabel('execution time')
axs[1].legend()

fig.tight_layout()

## Exercise 2. Numerical solution of a linear wave equation using a Chebyshev collocation spectral method.

In this exercise, we are interested in the numerical solution of an initial-value problem for a linear wave equation set in a two-dimensional domain,
$$
\frac{\partial^2u}{\partial t^2}(t,x,y)-\left(\frac{\partial^2u}{\partial x^2}(t,x,y)+\frac{\partial^2u}{\partial y^2}(t,x,y)\right)=0,\ (x,y)\in(-1,1)^2,\ t\in(0,+\infty),
$$
with homogeneous Dirichlet boundary conditions
$$
\forall t\in[0,+\infty),\ \forall y\in(-1,1),\ u(t,\pm1,y)=0\text{ and }\forall x\in(-1,1),\ u(t,x,\pm1)=0,
$$
and initial conditions
$$
\forall(x,y)\in(-1,1)^2,\ u(0,x,y)=e^{-40((x-0.4)^2+y^2)},\ \frac{\partial u}{\partial t}(0,x,y)=0,
$$
using a Chebyshev collocation spectral method in space and a leap-frog discretisation in time (this is the so-called [method of lines](https://en.wikipedia.org/wiki/Method_of_lines)). Note that the data for initial condition do not satisfy the boundary conditions in theory, but do so at the numerical level.

**Question.** Give the formula for the values of the second-order Chebyshev interpolation derivative of a function $u$ at the interior Chebyshev-Gauss-Lobatto points in terms of the discrete Chebyshev coefficients of the function.

**Answer.** Given a non-zero integer $N$, the Chebyshev-Gauss-Lobatto points are
$$
x_j=\cos\left(\frac{\pi j}{N}\right),\ j=0,\dots,N,
$$
and the values of the second-order Chebyshev interpolation derivative of a function $u$ at the interior points are given by
$$
\forall j\in\{1,\dots,N-1\},\ (I_Nu)''(x_j)=\frac{x_j}{(1-{x_j}^2)^{3/2}}\sum_{k=1}^{N-1}k\tilde{u}_k\sin\left(k\frac{j\pi}{N}\right)-\frac{1}{1-{x_j}^2}\sum_{k=0}^{N}k^2\tilde{u}_k\cos\left(k\frac{j\pi}{N}\right),
$$
the scalars $\tilde{u}_k$, $k=0,\dots,N$, being the discrete Chebyshev coefficients of $u$.

**Question.** Propose an algorithm with $O(N\log_2(N))$ complexity to compute these values using functions from the `fft` library of <tt>scipy</tt>.

**Answer.** To compute the value $(I_Nu)''(x_j)$, one first computes the discrete Chebyshev coefficients using a discrete cosine transform (and a possible normalisation) for a cost of $O(N\log_2(N))$. These coefficients are then multiplied to obtain the coefficients in the sums of sines and cosines appearing in the above formula for a cost of $O(N)$ operations. The two sums are then computed using an inverse discrete sine transform and an inverse discrete cosine transform, respectively, (and possible normalisations) for a cost of $O(N\log_2(N))$ each. Finally, the sum are added to one another, after a multiplication by their respective coefficients for a cost of two additions, four multiplications, two divisions and one square-root extraction. The overall cost of this algorithm is of $O(N\log_2(N))$ operations.

**Question.** Use this algorithm in a program solving the above initial-value problem up to time $T=10$ using a Chebyshev collocation spectral method for the space discretisation (choosing $N=32$ for instance) and a leap-frog scheme for the time discretisation. The choice of time-step length will be such that it satisfies a CFL-like condition of the form $\Delta t\leq C N^{-2}$, with $C$ a positive constant such that $C\leq 6.5$ (this condition follows from the determination of the stability region of the leap-frog and of the largest eigenvalue of the space discretisation operator). The result may be plotted as an animation.

In [4]:
# for animations
matplotlib.rcParams['animation.embed_limit']=2**128
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import animation
from IPython.display import HTML

In [ ]:
# spatial discretisation
N=32
x=np.cos(np.pi*np.arange(0,N+1)/N) # Chebyshev points in the x direction
y=x # Chebyshev points in the y direction

xx,yy=np.meshgrid(x,y)

# time discretisation
# enforcement of the CFL-like condition
C=6.
dt=C/(N**2)
T=10.
niter=int(T/dt)

vv=np.exp(-40*((xx-0.4)**2+yy**2)) # first initial condition
vvold=vv # second initial condition

nplots=niter+2
V=np.zeros((nplots,N+1,N+1))
V[0,:,:]=vv

# time-stepping using a leap-frog scheme
for n in range(0,niter+1):
    t=(n+1)*dt
        
    ii=np.arange(1,N) # the PDE is solved at the interior points

    uxx=np.zeros((N+1,N+1))
    for i in range(1,N): # 2nd derivative wrt x in each row
        v=vv[i,:]
        U=scp.fft.dct(v,type=1)/N
        W1=N*scp.fft.idst(np.arange(1,N)*U[1:-1],type=1)
        W2=N*scp.fft.idct(np.arange(0,N+1)**2*U,type=1)[ii]     
        uxx[i,ii]=x[ii]*W1/(1-x[ii]**2)**(3./2.)-W2/(1-x[ii]**2)
    
    uyy=np.zeros((N+1,N+1))
    for j in range(1,N): # 2nd derivative wrt y in each column
        v=vv[:,j]
        U=scp.fft.dct(v,type=1)/N
        W1=N*scp.fft.idst(np.arange(1,N)*U[1:-1],type=1)
        W2=N*scp.fft.idct(np.arange(0,N+1)**2*U,type=1)[ii]        
        uyy[ii,j]=y[ii]*W1/(1-y[ii]**2)**(3./2.)-W2/(1-y[ii]**2)

    vv,vvold=2*vv-vvold+dt**2*(uxx+uyy),vv
    V[n+1,:,:]=vv

# plot the result as an animation
fig,ax=plt.subplots(subplot_kw={"projection": "3d"})
ax.set_xlim((-1.,1))
ax.set_xlabel(r'$x$')
ax.set_ylim((-1.,1.))
ax.set_ylabel(r'$y$')
ax.set_zlim((-1.,1.))
ax.set_zlabel(r'$u(t,x,y)$')

def animate(i,zarray,frame):
    frame[0].remove()
    frame[0]=ax.plot_surface(xx,yy,zarray[i,:,:],rstride=1,cstride=1,cmap=cm.autumn,edgecolor='black')
    ax.set_title(r't='+f'{i*dt:.1f}')

frame=[ax.plot_surface(xx,yy,V[0,:,:],rstride=1,cstride=1,cmap=cm.autumn,edgecolor='black')]

anim=animation.FuncAnimation(fig,animate,nplots,fargs=(V,frame),interval=10)
HTML(anim.to_jshtml())

Second-order Chebyshev interpolation differentiation can be achieved using the matrix given by
$$
\left(D_N^{(2)}\right)_{jl}=\begin{cases}\dfrac{(-1)^{j+l}}{c_l}\dfrac{{x_j}^2+x_jx_l-2}{(1-{x_j}^2)(x_j-x_l)^2},&1\leq j\leq N-1,\ 0\leq l\leq N,\ j\neq l,\\-\dfrac{(N^2-1)(1-{x_j}^2)+3}{3(1-{x_j}^2)^2},&1\leq j,l\leq N-1,\ j=l,\\\dfrac{2}{3}\dfrac{(-1)^l}{c_l}\dfrac{(2N^2+1)(1-x_l)-6}{(1-x_l)^2},&j=0,\ 1\leq l\leq N,\\\dfrac{2}{3}\dfrac{(-1)^{l+N}}{c_l}\dfrac{(2N^2+1)(1+x_l)-6}{(1+x_l)^2},&j=N,\ 0\leq l\leq N-1\\\dfrac{N^4-1}{15},&j=l=0\text{ or }N,\end{cases}
$$
where
$$
c_j=\begin{cases}2,&j=0\text{ or }N,\\1,&j=1,\dots,N-1.\end{cases}
$$
To ease the implementation in this notebook, we note that this matrix can be obtained as the square of the first-order Chebyshev differentiation matrix, at the cost of $O(N^3)$ operations, the latter having the form already seen in the preceding exercise:
$$
\left(D_N^{(1)}\right)_{jl}=\begin{cases}\dfrac{2N^2+1}{6}&j=l=0,\\-\dfrac{x_j}{2(1-{x_j}^2)}&1\leq j,l\leq N-1,\ j=l\\\dfrac{c_i}{c_j}\dfrac{(-1)^{j+l}}{x_j-x_l}&1\leq j,l\leq N-1,\ j\neq l,\\-\dfrac{2N^2+1}{6}&j=l=N.\end{cases}
$$
**Question.** Modify the previous program to make use of matrix multiplications instead of fast Fourier transforms.

In [ ]:
# spatial discretisation
N=32
x=np.cos(np.pi*np.arange(0,N+1)/N) # Chebyshev points in the x direction
y=x # Chebyshev points in the y direction

xx,yy=np.meshgrid(x,y)

# time discretisation
# enforcement of the CFL-like condition
C=6.
dt=C/(N**2)
T=10.
niter=int(T/dt)

vv=np.exp(-40*((xx-0.4)**2+yy**2)) # first initial condition
vvold=vv # second initial condition

# computation of the Chebyshev differentiation matrices
# first-order differentiation matrix
c=np.ones(N+1)
c[0],c[N]=2.,2.
c=c*(-1.)**np.arange(0,N+1)
c=c.reshape(N+1,1)
X=np.tile(x.reshape(N+1,1),(1,N+1))
D=np.dot(c,1./c.T)/(X-X.T+np.eye(N+1)) # off-diagonal entries
D=D-np.diag(D.sum(axis=1)) # diagonal entries
# second-order differentiation matrix
D2=np.dot(D,D)

nplots=niter+2
V=np.zeros((nplots,N+1,N+1))
V[0,:,:]=vv

# time-stepping using a leap-frog scheme
for n in range(0,niter+1):
    t=(n+1)*dt
        
    ii=np.arange(1,N) # the PDE is solved at the interior points

    uxx=np.zeros((N+1,N+1))
    for i in range(1,N): # 2nd derivative wrt x in each row
        uxx[i,ii]=np.dot(D2[1:N,1:N],vv[i,ii])
    
    uyy=np.zeros((N+1,N+1))
    for j in range(1,N): # 2nd derivative wrt y in each column    
        uyy[ii,j]=np.dot(D2[1:N,1:N],vv[ii,j])

    vv,vvold=2*vv-vvold+dt**2*(uxx+uyy),vv
    V[n+1,:,:]=vv

# plot the result as an animation
fig,ax=plt.subplots(subplot_kw={"projection": "3d"})
ax.set_xlim((-1.,1))
ax.set_xlabel(r'$x$')
ax.set_ylim((-1.,1.))
ax.set_ylabel(r'$y$')
ax.set_zlim((-1.,1.))
ax.set_zlabel(r'$u(t,x,y)$')

def animate(i,zarray,frame):
    frame[0].remove()
    frame[0]=ax.plot_surface(xx,yy,zarray[i,:,:],rstride=1,cstride=1,cmap=cm.autumn,edgecolor='black')
    ax.set_title(r't='+f'{i*dt:.1f}')

frame=[ax.plot_surface(xx,yy,V[0,:,:],rstride=1,cstride=1,cmap=cm.autumn,edgecolor='black')]

anim=animation.FuncAnimation(fig,animate,nplots,fargs=(V,frame),interval=10)
HTML(anim.to_jshtml())